# Setup

## Imports

In [1]:
import os
import json

In [2]:
from snowflake.snowpark import Session

In [3]:
import snowflake.snowpark.modin.pandas as pd

## Session

In [5]:
my_dir = os.getcwd()
connection_parameters = json.load(open('/Users/skhara/Documents/Code/creds.json'))
session = Session.builder.configs(connection_parameters).create()

session.use_database('DEMO_DB')
session.use_schema('PUBLIC')

In [6]:
pd.session = session

# Testing 1 - simple

## Load Data

In [7]:
%%time
application_record_sdf = pd.read_snowflake('APPLICATION_RECORD')
credit_record_sdf = pd.read_snowflake('CREDIT_RECORD')
print('Application table size\t: ', application_record_sdf.count(), 
      '\nCredit table size\t: ', credit_record_sdf.count())

Application table size	:  ID                     438557
CODE_GENDER            438557
FLAG_OWN_CAR           438557
FLAG_OWN_REALTY        438557
CNT_CHILDREN           438557
AMT_INCOME_TOTAL       438557
NAME_INCOME_TYPE       438557
NAME_EDUCATION_TYPE    438557
NAME_FAMILY_STATUS     438557
NAME_HOUSING_TYPE      438557
DAYS_BIRTH             438557
DAYS_EMPLOYED          438557
FLAG_MOBIL             438557
FLAG_WORK_PHONE        438557
FLAG_PHONE             438557
FLAG_EMAIL             438557
OCCUPATION_TYPE        304354
CNT_FAM_MEMBERS        438557
dtype: int32 
Credit table size	:  ID                1048575
MONTHS_BALANCE    1048575
STATUS            1048575
dtype: int32
CPU times: user 95.8 ms, sys: 24.8 ms, total: 121 ms
Wall time: 7.49 s


In [8]:
len(application_record_sdf)

438557

In [9]:
application_record_sdf.head(5)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1


## Transform Data

In [10]:
%%time
credit_record_sdf['CNT_LATE'] = credit_record_sdf['STATUS'].apply(lambda x: 1 if x in ['2', '3', '4', '5'] else 0)

convert_dtype is ignored in Snowflake backend.


CPU times: user 81.6 ms, sys: 40.4 ms, total: 122 ms
Wall time: 28.2 s


In [11]:
credit_record_sdf.dtypes

ID                 int64
MONTHS_BALANCE     int64
STATUS            object
CNT_LATE          object
dtype: object

In [12]:
%%time
credit_temp = credit_record_sdf.groupby(['ID']).sum()['CNT_LATE'].reset_index()

CPU times: user 10.7 ms, sys: 2.93 ms, total: 13.6 ms
Wall time: 594 ms


In [13]:
credit_record_sdf['CNT_LATE'].unique()

array([0, 1])

In [14]:
credit_temp.head(5)

,ID,CNT_LATE
0,5001711,0.0
1,5001712,0.0
2,5001713,0.0
3,5001714,0.0
4,5001715,0.0


In [23]:
credit_temp.dtypes

ID            int64
CNT_LATE    float64
dtype: object

### While doing .apply() here it makes 'TARGET' column as Object. Need to convert to type Int

In [24]:
credit_temp['TARGET'] = credit_temp['CNT_LATE'].apply(lambda x: 1 if x > 0 else 0)
credit_temp.dtypes

ID            int64
CNT_LATE    float64
TARGET       object
dtype: object

In [25]:
%%time
credit_temp['TARGET'] = credit_temp['CNT_LATE'].apply(lambda x: 1 if x > 0 else 0).astype('int')
credit_cleaned = credit_temp.drop('CNT_LATE', axis=1).dropna()

CPU times: user 77.6 ms, sys: 11.8 ms, total: 89.4 ms
Wall time: 1.88 s


In [26]:
credit_temp['CNT_LATE'].unique()

array([ 0.,  5.,  1.,  2.,  3., 33.,  6.,  9.,  7., 15., 20., 11.,  4.,
       13., 22., 38., 14., 12.,  8., 10., 18., 39., 17., 40., 27., 21.,
       16., 48., 26., 19., 24., 31., 45., 43., 47., 23., 35.])

In [27]:
credit_temp['TARGET'].unique()

array([0, 1], dtype=int8)

In [28]:
arr = credit_cleaned['TARGET'].unique()

In [29]:
type(arr[1])

numpy.int8

In [30]:
%%time
credit_cleaned.isna().sum()

CPU times: user 32.7 ms, sys: 6.31 ms, total: 39 ms
Wall time: 392 ms


ID        0
TARGET    0
dtype: int8

In [ ]:
credit_cleaned.head(5)

In [ ]:
credit_cleaned.dtypes

In [ ]:
temp = credit_cleaned.limit(5).to_pandas()
temp

In [ ]:
temp.dtypes

In [ ]:
%%time
joined_sdf = application_record_sdf.merge(credit_cleaned, left_on='ID', right_on='ID', how='inner')

In [ ]:
%%time
joined_sdf.head(5)

In [ ]:
cols_numerical = ['AMT_INCOME_TOTAL', 'DAYS_EMPLOYED', 'FLAG_MOBIL', 'CNT_FAM_MEMBERS', 'TARGET']
cols_categorical = ['CODE_GENDER', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE']
joined_sdf = joined_sdf[cols_numerical+cols_categorical]

# General Testing on 500K Rows Table

## Where

Observations ->
1. .where doesn't work when column given inside and run on entire DF.

#### ** Where without column

In [16]:
%%time
application_record_sdf.where(application_record_sdf['AMT_INCOME_TOTAL'] > 200000)

CPU times: user 35.4 ms, sys: 3.94 ms, total: 39.3 ms
Wall time: 421 ms


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,None,None,None,None,None,427500.0,None,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,427500.0,None,None,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,270000.0,None,None,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,270000.0,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None
438553,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None
438554,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None
438555,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None


In [17]:
%%time
application_record_sdf['AMT_INCOME_TOTAL'].where(application_record_sdf['AMT_INCOME_TOTAL'] > 200000).head()

CPU times: user 29.9 ms, sys: 3.03 ms, total: 32.9 ms
Wall time: 387 ms


0    427500.0
1    427500.0
2         NaN
3    270000.0
4    270000.0
Name: AMT_INCOME_TOTAL, dtype: float64

In [18]:
%%time
application_record_sdf[application_record_sdf['AMT_INCOME_TOTAL'] > 200000].head()

CPU times: user 16 ms, sys: 2.03 ms, total: 18 ms
Wall time: 252 ms


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,None,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
5,5008810,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


## Merge

In [ ]:
%%time
df1 = application_record_sdf[['ID', 'AMT_INCOME_TOTAL']]
df2 = application_record_sdf[['ID', 'NAME_EDUCATION_TYPE']]
merged_data = df1.merge(df2, on='ID')

In [ ]:
merged_data.head()

## Sub

Subtract 10000 from DF compare gender salaries

In [ ]:
%%time
df1 = application_record_sdf[['CODE_GENDER','AMT_INCOME_TOTAL']]
df1 = df1.groupby('CODE_GENDER').mean()['AMT_INCOME_TOTAL'].astype('int')

In [ ]:
%%time
df1 = application_record_sdf[['CODE_GENDER','AMT_INCOME_TOTAL']]
df1 = df1.groupby('CODE_GENDER').mean()['AMT_INCOME_TOTAL'].astype('int')
df1

In [ ]:
%%time
df1

In [ ]:
print(df1)

In [ ]:
print(df1.head())

In [ ]:
%%time
df2 = application_record_sdf[['CODE_GENDER','AMT_INCOME_TOTAL']]
df2['AMT_INCOME_TOTAL'] = df2['AMT_INCOME_TOTAL'].sub(10000)

In [ ]:
%%time
df2 = df2.groupby('CODE_GENDER').mean()['AMT_INCOME_TOTAL'].astype('int')
print(df2.head())

In [ ]:
df1 - df2

## Groupby

Observations ->
1. Cannot get groups

#### ** Groups

In [ ]:
type(application_record_sdf)

In [ ]:
grouped_df = application_record_sdf.groupby('CODE_GENDER')
grouped_df.groups.keys() #doesnt work yet

#### Groupby + Agg

In [ ]:
%%time
application_record_sdf.groupby('NAME_EDUCATION_TYPE')['AMT_INCOME_TOTAL'].agg(['min', 'max', 'mean'])

In [ ]:
%%time
application_record_sdf.groupby('NAME_EDUCATION_TYPE').agg({'CNT_FAM_MEMBERS': 'max',
                                                           'AMT_INCOME_TOTAL': 'mean'})

In [ ]:
temp = application_record_sdf.groupby('NAME_EDUCATION_TYPE').agg({'CNT_FAM_MEMBERS': 'max',
                                                           'AMT_INCOME_TOTAL': 'mean'})

temp

#### Multi-level Grouping

In [ ]:
%%time
application_record_sdf.groupby(['NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS'])['AMT_INCOME_TOTAL'].mean().reset_index()

## ** Transform and Sort

In [ ]:
application_record_sdf.head()

In [ ]:
%%time
temp = application_record_sdf.copy()

In [ ]:
%%time
temp['AGE_YEARS'] = -temp['DAYS_BIRTH']

In [ ]:
%%time
temp['AGE_YEARS'] = temp['AGE_YEARS']/365

In [ ]:
%%time
temp['AGE_YEARS'] = temp['AGE_YEARS'].astype(int)

In [ ]:
%%time
temp = temp.sort_values(by='AGE_YEARS', ascending=False)

In [ ]:
temp.head()

## Pivot Table & Multiple Aggs

In [ ]:
%%time
application_record_sdf['AMT_INCOME_TOTAL'] = application_record_sdf['AMT_INCOME_TOTAL'].astype('int')
application_record_sdf.pivot_table(values='AMT_INCOME_TOTAL',
                                   index='NAME_EDUCATION_TYPE',
                                   columns='NAME_FAMILY_STATUS',
                                   aggfunc=['mean', 'max'])

## ** Conditional Assignment

In [ ]:
temp = application_record_sdf.copy()

In [ ]:
temp.head()

In [ ]:
%%time
temp['PERSON_DESC'] = ''
temp.loc[(temp['DAYS_BIRTH'] / -365 < 30) & (temp['AMT_INCOME_TOTAL'] > 200000), 'PERSON_DESC'] = 'Young High Earner'
temp.loc[(temp['DAYS_BIRTH'] / -365 > 30) & (temp['AMT_INCOME_TOTAL'] <= 200000), 'PERSON_DESC'] = 'Older Low Earner'

In [ ]:
temp.head()

## ** Merging and Sub Two Cols


In [ ]:
application_record_sdf

In [ ]:
%%time
df1 = application_record_sdf[['ID', 'DAYS_BIRTH']]
df2 = application_record_sdf[['ID', 'DAYS_EMPLOYED']]

In [ ]:
%%time
merged_data = df1.merge(df2, on='ID', how='left').fillna(0)

In [ ]:
%%time
merged_data['DAYS_BEFORE_JOB'] = merged_data['DAYS_EMPLOYED'] - merged_data['DAYS_BIRTH']

In [ ]:
%%time
merged_data.head()

## ** Resampling (Requires Datetime Conversion):


In [ ]:
%%time
df1 = application_record_sdf.copy()

In [ ]:
%%time
df1['DAYS_BIRTH'] = -df1['DAYS_BIRTH']

In [ ]:
%%time
df1['DOB'] = pd.to_datetime(df1['DAYS_BIRTH'], origin='1900-01-01', unit='D')

In [ ]:
df1.head()

In [ ]:
%%time
df1['DOB_YEAR'] = df1['DOB'].dt.year

In [ ]:
%%time
df1.groupby('DOB_YEAR')['DOB'].count().sort_values(ascending=False)

#### Pandas

In [ ]:
df1_pandas = df1.to_pandas()

In [ ]:
len(df1_pandas)

In [ ]:
%%time
df1_pandas['DOB'].dt.year

## Column Rename, Head, and Filter

In [ ]:
%%time
df1 = application_record_sdf.copy()

In [ ]:
df1.head()

In [ ]:
%%time
df1 = df1.rename(columns={'FLAG_EMAIL': 'HAS_EMAIL'})

In [ ]:
%%time
df1.loc[df1['HAS_EMAIL'] == 1].head()